# csdid with RC/Panel

First. Some other tools. Please copy
[this](https://friosavila.github.io/stpackages/fra/fra.ado) file, run
it, and type:

``` stata
** just for data example
ssc install frause
** for installing csdid2
fra install fra
fra install csdid2
```

Now the example.

First Run csdid with panel data

In [1]:
frause mpdta, clear
set seed 1
csdid lemp, ivar( countyreal) gvar(first) time(year)

(Written by R.              )
............
Difference-in-difference with Multiple Time Periods

                                                         Number of obs = 2,500
Outcome model  : regression adjustment
Treatment model: none
------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
g2004        |
 t_2003_2004 |  -.0105032    .023251    -0.45   0.651    -.0560744    .0350679
 t_2003_2005 |  -.0704232   .0309848    -2.27   0.023    -.1311522   -.0096941
 t_2003_2006 |  -.1372587   .0364357    -3.77   0.000    -.2086713   -.0658461
 t_2003_2007 |  -.1008114   .0343592    -2.93   0.003    -.1681542   -.0334685
-------------+----------------------------------------------------------------
g2006        |
 t_2003_2004 |   .0065201   .0233268     0.28   0.780    -.0391996    .0522398
 t_2004_2005 |  -.00275

Then, drop at randome 10% of the data, so technically have unbalanced
panel. It can be estimated using repeated crossection

In [2]:
drop if runiform()<.1
** as Panel with unbalanced Data
csdid lemp lpop,  ivar( countyreal) gvar(first) time(year)

(239 observations deleted)
Panel is not balanced
Will use observations with Pair balanced (observed at t0 and t1)
............
Difference-in-difference with Multiple Time Periods

                                                         Number of obs = 2,227
Outcome model  : least squares
Treatment model: inverse probability
------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
g2004        |
 t_2003_2004 |  -.0106321   .0213814    -0.50   0.619     -.052539    .0312747
 t_2003_2005 |  -.0997826   .0338647    -2.95   0.003    -.1661562   -.0334089
 t_2003_2006 |   -.164467   .0421525    -3.90   0.000    -.2470845   -.0818495
 t_2003_2007 |  -.1372957    .029445    -4.66   0.000    -.1950069   -.0795845
-------------+----------------------------------------------------------------
g2006        |
 t_2003_200

In [3]:
** As RC
csdid lemp lpop,  cluster( countyreal) gvar(first) time(year)

............
Difference-in-difference with Multiple Time Periods

                                                         Number of obs = 2,261
Outcome model  : least squares
Treatment model: inverse probability
                           (Std. err. adjusted for 500 clusters in countyreal)
------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
g2004        |
 t_2003_2004 |  -.0126297   .0345604    -0.37   0.715    -.0803669    .0551075
 t_2003_2005 |  -.0913197   .0384773    -2.37   0.018    -.1667338   -.0159055
 t_2003_2006 |  -.2070359    .049318    -4.20   0.000    -.3036974   -.1103744
 t_2003_2007 |   -.136554   .0317776    -4.30   0.000     -.198837   -.0742711
-------------+----------------------------------------------------------------
g2006        |
 t_2003_2004 |  -.0098464   .0303239    -0.32 

The second one produces odd results, which is not unexpected.

Now using the residuals Idea:

In [4]:
** Getting residuals wo year
qui:reghdfe lemp, abs(countyreal) resid
** AS RC but with residuals
csdid _reghdfe_resid lpop, cluster( countyreal) gvar(first) time(year)

............
Difference-in-difference with Multiple Time Periods

                                                         Number of obs = 2,260
Outcome model  : least squares
Treatment model: inverse probability
                           (Std. err. adjusted for 499 clusters in countyreal)
------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
g2004        |
 t_2003_2004 |  -.0235323   .0248084    -0.95   0.343     -.072156    .0250913
 t_2003_2005 |  -.0901505   .0312695    -2.88   0.004    -.1514376   -.0288633
 t_2003_2006 |  -.1579142   .0380608    -4.15   0.000     -.232512   -.0833165
 t_2003_2007 |   -.119084   .0323402    -3.68   0.000    -.1824697   -.0556984
-------------+----------------------------------------------------------------
g2006        |
 t_2003_2004 |  -.0141947    .024901    -0.57 

In [5]:
** Getting residuals
qui:reghdfe lemp, abs(countyreal year) resid
** AS RC but with residuals
csdid _reghdfe_resid lpop, cluster( countyreal) gvar(first) time(year)

............
Difference-in-difference with Multiple Time Periods

                                                         Number of obs = 2,260
Outcome model  : least squares
Treatment model: inverse probability
                           (Std. err. adjusted for 499 clusters in countyreal)
------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
g2004        |
 t_2003_2004 |   -.022436   .0249588    -0.90   0.369    -.0713544    .0264824
 t_2003_2005 |  -.0898449   .0312509    -2.87   0.004    -.1510956   -.0285942
 t_2003_2006 |  -.1577356   .0380741    -4.14   0.000    -.2323594   -.0831118
 t_2003_2007 |  -.1192472   .0322187    -3.70   0.000    -.1823947   -.0560998
-------------+----------------------------------------------------------------
g2006        |
 t_2003_2004 |  -.0139274   .0249804    -0.56 

I think this produces much better results! Not sure how would this
translate in other frameworks, tho.